# #50 Create a simple cantilever line beam model
<i>Creates and solves a steel beam cantilver model</i>
*** 


In [ ]:
# Input Params
length = 5.0 #m		     Length of beam
point_load = 10.0  #kN   Concentrated load on cantilever tip

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI_22_0 import *
lusas = get_lusas_modeller()
if lusas.existsDatabase():
    raise Exception("This script will create a new model. Please save and close the current model and try again")

<div class="alert alert-block alert-info">
<b>Tip:</b> When interacting with Modeller externally, as we are here. A significant speed up can be achieved by disabling the user interface. In v22 and later this can be done with the <b>enableUI</b> function, in earlier versions the application must be set to be invisible. Take care to restore the visibility and watch out for exceptions in the code that can leave Modeller running in the background or disabled. See the code below to ensure this is properly restored.
</div>

In [ ]:
do_speed_up = False # To speed up interaction with Modeller from python set this to True

if do_speed_up: 
    if lusas.getMajorVersionNumber() >= 22:
        lusas.enableUI(False)
    else:
        lusas.setVisible(False)

In [ ]:
# Create a new model
lusas.newProject("Structural", "Cantilever Line Beam.mdl")
# Get a reference to the model database
db = lusas.getDatabase()
# Set the analysis category & vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

In [ ]:
''' Create a line'''
# Get the geometry data object
geomData = lusas.geometryData()
# Set the defaults
geomData.setAllDefaults()
# Set the coordinates of the first point
geomData.addCoords(0, 0, 0)
# Set the coordinates of the second point
geomData.addCoords(length, 0, 0)
# Create the line object from the geometry data
objs = db.createLine(geomData) # Modeller returns an object set containing all the lines created, in this case we only have 1.
# Get the lines
lines = objs.getObjects("Lines")
# Get the single line from the returned array
beamLine = lines[0]

In [ ]:
''' Create a mesh attribute'''
meshAttr = db.createMeshLine("Beam Mesh")
# Set the element type and size of elements BMI21 = 2Node thick beam elements, 1 = 1m length
meshAttr.setSize("BMI21", 1)

In [ ]:
''' Create a geometric attribute'''
geomAttr = db.createGeometricLine("Beam Geometry")
# Set the element type
geomAttr.setValue("elementType", "3D Thick Beam")
# Set the beam section properties for the section library
geomAttr.setFromLibrary("UK Sections", "Universal Beams (BS4)", "406x178x74kg UB", 0, 0, 0)

In [ ]:
''' Create a material attribute'''
materialAttr = db.createIsotropicMaterial("Steel", 209_000_000, 0.3, 7.8)

In [ ]:
''' Create a support attribute (fixed) '''
fixedSupport = db.createSupportStructural("Fixed")
# set the freedoms F=Free, R=Restrained
fixedSupport.setStructural("R", "R", "R", "R", "R", "R", "F", "F", "F")

In [ ]:
''' Create a point load attribute '''
loadAttr = db.createLoadingConcentrated("Point load")
loadAttr.setConcentrated(0.0, 0.0, -point_load)

In [ ]:
''' Assign the attributes to the geometry '''
# get the assignment object
assignment = lusas.assignment()
# set the defaults
assignment.setAllDefaults()

# Assign the mesh
meshAttr.assignTo(beamLine, assignment)
# Assign the geometry
geomAttr.assignTo(beamLine, assignment)
# Assign the material
materialAttr.assignTo(beamLine, assignment)
# Assign the loading
loadAttr.assignTo(beamLine.getEndPoint(), assignment)

''' Assign the supports to the points of the line '''
# Assign the fixed support to the first point
fixedSupport.assignTo(beamLine.getStartPoint(), assignment)


In [ ]:
# Mesh the model, ready for analysis
db.resetMesh()
db.updateMesh()
# Set the view
lusas.view().setIsometric()

In [ ]:
# Solve the analysis
db.getAnalysis("Analysis 1").solve(True)
db.openAllResults(True, True)

In [ ]:
# Display the deformed contours
lusas.view().insertContoursLayer()
lusas.view().contours().setResultsTransformNone()
lusas.view().contours().setResults("Displacement", "DZ")
lusas.view().contours().setShowDeformed(True)
lusas.view().insertAnnotationLayer()
lusas.view().attributes().setShowDeformed("Geometric", True)

<div class="alert alert-block alert-warning">
<b>Note:</b> If the code fails before this point the cell below must be run manually to restore the user interface
</div>

In [ ]:
if lusas.getMajorVersionNumber() >= 22:
    lusas.enableUI(True)
else:
    lusas.setVisible(True)